# GPS Simulation Project

### Haversine Function

In [0]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


# Sample for filling the function print( distance((lat1, long1), (lat2, long2)) )

### Reading points from digital maps
![image.png](attachment:image.png)

In [2]:
from pykml import parser
import pandas as pd

filename='KirunaUmeaStation.kml'
with open(filename) as f:
    folder = parser.parse(f).getroot().Document.Folder

plnm = []
cordi = []
for pm in folder.Placemark:
    plnm1=pm.name
    plcs1=pm.Point.coordinates
    plnm.append(plnm1.text)
    cordi.append(plcs1.text)
    
db=pd.DataFrame()
db['place_name']=plnm
db['cordinates']=cordi

def dump(obj):
    for attr in dir(obj):
        if hasattr( obj, attr ):
            print( "obj.%s = %s" % (attr, getattr(obj, attr)))
#the kml output plnm1 and plcs1 are objects actually which can be viewed using the dump function from [SO](https://blender.stackexchange.com/questions/1879/is-it-possible-to-dump-an-objects-properties-and-methods)

db['Longitude'], db['Latitude'],db['value'] = zip(*db['cordinates'].apply(lambda x: x.split(',', 2)))
db.drop(['value','cordinates'],axis=1,inplace=True)

db

ImportError: ignored

### Creating appropirate dataBase for the calculating points

In [0]:
#db1 = db[['Longitude','Latitude']]
db.Longitude.str.replace('\n','')  # Eliminating the \n from the Longitude
db['Lon_int'] = db.Longitude.astype(float)
db['Lat_int'] = db.Latitude.astype(float) 
db1 = db[['place_name','Lat_int','Lon_int']]
db1.info

<bound method DataFrame.info of       place_name    Lat_int    Lon_int
0   01. Point 01  67.870525  20.197191
1    02. Point 2  67.868731  20.198698
2    03. Point 3  67.862818  20.205054
3    04. Point 4  67.859186  20.203772
4    05. Point 5  67.858910  20.204381
5    06. Point 6  67.856564  20.213314
6    07. Point 7  67.851906  20.217706
7    08. Point 8  67.849248  20.219541
8    09. Point 9  67.845861  20.220871
9   10. Point 10  67.840362  20.223143
10  11. Point 11  67.837863  20.227511
11  12. Point 12  67.836849  20.230552
12  13. Point 13  67.834565  20.235902
13  14. Point 14  67.829823  20.237574
14  15. Point 15  67.825095  20.240494>

### Calculating the distance between any two sequenced points

In [0]:
[m,n] = db1.shape
 
Distances = []
for i in range(m-1):
    TempResult = distance((db1.iloc[i,1],db1.iloc[i,2]),(db1.iloc[i+1,1],db1.iloc[i+1,2]))
    Distances.append(["Track {}".format(i),TempResult])
    
#Converting Distances List to Pandas Dataframe
dbDistances = pd.DataFrame(Distances)

### Representing the distance on each track (km)

In [0]:
dbDistances

,0,1
0,Track 0,0.209274
1,Track 1,0.709371
2,Track 2,0.407416
3,Track 3,0.039915
4,Track 4,0.456290
5,Track 5,0.549637
6,Track 6,0.305446
7,Track 7,0.380733
8,Track 8,0.618796
9,Track 9,0.332866


### Total distance passed

In [0]:
TotalDistance = dbDistances.sum()
TotalDistance

0    Track 0Track 1Track 2Track 3Track 4Track 5Trac...
1                                              5.59073
dtype: object

# GPS simulation 


### Calculating angle  between two sequential points which is called BEARING
#### Function of Bearing (Horisontal Bearing)

In [0]:
def calcBearing(lat1, lon1, lat2, lon2):
    dLon = lon2 - lon1
    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) \
        - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)
    return math.atan2(y, x)

In [0]:
Bearings = []
for i in range(m-1):
    TempBearing = calcBearing(db1.iloc[i,1],db1.iloc[i,2],db1.iloc[i+1,1],db1.iloc[i+1,2])
    Bearings.append(["Track {}".format(i),TempBearing])

### Representing the bear on each track

In [0]:
Bearings

[['Track 0', 2.879887637092737],
 ['Track 1', 2.817182585184142],
 ['Track 2', -3.0327109170377495],
 ['Track 3', 2.542115964944342],
 ['Track 4', 2.2804876954270807],
 ['Track 5', 2.86358921756057],
 ['Track 6', 2.9371323663653004],
 ['Track 7', 3.0254950506856613],
 ['Track 8', 3.021606293604686],
 ['Track 9', 2.673834559716899],
 ['Track 10', 2.4289437803808753],
 ['Track 11', 2.551622423225236],
 ['Track 12', 3.0426161083292302],
 ['Track 13', 2.9721148316833372]]